In [5]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import layout
from bokeh.io import push_notebook, show, output_notebook
import numpy as np
from tensorflow.keras.models import load_model
import sys
import pandas as pd
import paho.mqtt.client as mqtt
import json
import datetime
import socket
import threading
import time
from math import *
import random




def transformTimestamps(timestamp):
    transform_time = timestamp + 946656000
    date = datetime.datetime.fromtimestamp(transform_time)
    time_str = date.strftime('%Y-%m-%d %H:%M:%S')
    return time_str

# 更新函數，每次呼叫時更新數據
def update(message):
    global x, y_data, grap_id, x_record, rawData, record_count, max_data_count, stop_time_seconds
    
    # 解碼 MQTT 數據
    if test_mode:
        data = message
    else:
        payload = message.payload.decode()
        data = json.loads(payload)

    
    mqData = np.array(data)


    times = mqData[:, 0]
    channel1 = mqData[:, 1]
    channel2 = mqData[:, 2]
    channel3 = mqData[:, 3]
    channel4 = mqData[:, 4]
    channel5 = mqData[:, 5]
    channel6 = mqData[:, 6]
    channel7 = mqData[:, 7]
    channel8 = mqData[:, 8]
    channel9 = mqData[:, 9]
    channel10 = mqData[:, 10]
    channel11 = mqData[:, 11]
    channel12 = mqData[:, 12]



    



    

    # 将时间戳转换为本地时间元组
    # new_times = np.vectorize(transformTimestamps)(times)
    # print('times:', new_times)

    rawData.append(mqData)


    
    # 使用模型进行预测
    ranges = np.array([[1.7, 2], [6.7, 7], [-7, -6.6], [-4, 2], [-2, 2], [-8, -3], [0, 0.3], [9, 9.3], [-4.3, -4], [0, 5], [-2, 2], [-6, -4]])
    # 使用 numpy.random.uniform 生成具有不同範圍的數據
    dataForTest = np.column_stack([np.random.uniform(low=low, high=high, size=30) for low, high in ranges])
    print('-----------------')
    dataForTest = dataForTest[np.newaxis, ...]
    # print('transforming test random data shape:', dataForTest.shape)
    print('data for test:', dataForTest)


    # 使用訓練數據展示結果
    dataForTrain = trained_data_x[record_count * data_length : record_count * data_length + data_length, :]
    dataForTrain = dataForTrain[np.newaxis, ...]
    # print('transforming train data shape:', dataForTrain.shape)
    # print('data for train:', dataForTrain)

    fifoData = np.concatenate(rawData, axis=0)
    fifoData = fifoData[record_count : record_count + window_size, :]
    fifoData = fifoData[:, 1:]
    fifoData = fifoData[np.newaxis, ...]
    fifoData = fifoData.astype(float)
    print('transforming fifo data shape:', fifoData.shape)
    print('fifo:', fifoData)
    y_pred = model.predict(dataForTest)

    predictions.append(y_pred[0])
    print('predictions:', predictions)
    print('------------------')

    # 将预测结果转换为 NumPy 数组
    predictionsNp = np.array(predictions)
    print('predictoinNp:', predictionsNp)
    print('------------------')
    print('predictions shape:', predictionsNp.shape)
    print(predictionsNp)
    # 提取预测类别的最大概率 >0.5 => 1, <0.5 => 0
    predicted_classes = predictionsNp[-1]
    print(predicted_classes)
    # 只會有一個結果, 值為 0 或者 1
    print(predicted_classes[0])
    prediction_label = round(predicted_classes[0])
    print('predicted_classes', predicted_classes)
    print('prediction_label', prediction_label)
    x_record += 1


    # 更新數據源的數據
    grap_id += data_length

    record_count = record_count + 1


    # 更新數據源的數據
    grap_id += data_length
    source1.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel1})
    source2.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel2})
    source3.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel3})
    source4.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel4})
    source5.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel5})
    source6.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel6})
    source7.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel7})
    source8.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel8})
    source9.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel9})
    source10.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel10})
    source11.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel11})
    source12.stream({'x':np.arange(grap_id, grap_id + data_length), 'y':channel12})
    source13.stream({'x':np.arange(x_record, x_record + 1), 'y': [prediction_label]})


    if record_count % 1000 == 0:
        exportCSV()

    push_notebook()

    



def exportCSV():
    global rawData
    csvData = np.concatenate(rawData, axis=0)
    timestamps = csvData[:, 0]
    ax1 = csvData[:, 1]
    ay1 = csvData[:, 2]
    az1 = csvData[:, 3]
    gx1 = csvData[:, 4]
    gy1 = csvData[:, 5]
    gz1 = csvData[:, 6]

    ax2 = csvData[:, 7] 
    ay2 = csvData[:, 8]
    az2 = csvData[:, 9]
    gx2 = csvData[:, 10]
    gy2 = csvData[:, 11]
    gz2 = csvData[:, 12]

    # 将时间戳转换为本地时间元组
    # new_times = np.vectorize(transformTimestamps)(timestamps)

    # dictionary of lists
    dict = {'Time': timestamps, 'Ax1': ax1, 'Ay1': ay1, 'Az1': az1, 'Gx1': gx1, 'Gy1': gy1, 'Gz1': gz1, 'Ax2': ax2, 'Ay2': ay2, 'Az2': az2, 'Gx2': gx2, 'Gy2': gy2, 'Gz2': gz2}
        
    df = pd.DataFrame(dict)
    filename = 'output.csv'
    df.to_csv(filename, index=False)


def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
    else:
        print("Failed to connect, return code %d\n", rc)

def on_message(client, userdata, message):
    print("Received data from esp32 via MQTT")
    update(message)
    # print("Received message on topic %s: %s" % (message.topic, message.payload.decode()))



def broadcast(command):
    """ Send a command to all connected clients. """
    with clients_lock:  # Acquire lock to ensure list is not modified while iterating
        for client in clients:
            try:
                client.send(command.encode('utf-8'))  # Send the command to each client
            except socket.error as e:  # Handle potential socket errors
                print(f'Error sending to client: {e}')

def client_thread(client_socket, client_address):
    global data_buffer
    """ Handle communication with a connected client. """
    print(f'New connection from {client_address}') # used to check the connection is created or not
    # Add the new client to the clients list in a thread-safe manner
    with clients_lock:
        clients.append(client_socket)
    try:
        while True:  # Continuously receive data from the client
            imu_data = client_socket.recv(164000).decode('utf-8')

            print(imu_data)
            imu_json = json.loads(imu_data)

            if imu_json['func'] == 'log':
                print('message from ESP32: ', imu_json['msg'])
            elif imu_json['func'] == 'visualization':
                update(imu_json['data'])
            # if not imu_data :  # If no data is received, it means the client has disconnected
            #     break
            # elif Special_Command == "Real_Time_Detection":
            #     # print('Receiving data:', imu_data)
            #     update([imu_data])
            #     data_buffer.append(imu_data)
            # else:

            # Here you could call broadcast() if needed

    except socket.error as e:  # Handle any exceptions that occur within the thread
        print(f'Error with client {client_address}: {e}')
    finally:
        # When the client disconnects, remove it from the clients list and close the socket
        with clients_lock:
            clients.remove(client_socket)
        client_socket.close()
        print(f'Connection with {client_address} closed')

def connect_mqtt_server():
    # 连接MQTT代理
    # broker_address = "broker.emqx.io"
    broker_address = "127.0.0.1"
    # broker_address = "broker.mqttdashboard.com"
    # broker_address = "test.mosquitto.org"
    port = 1883
    username = "ESP32_Monitor"
    password = "123"

    client = mqtt.Client()
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.on_message = on_message


    client.connect(broker_address, port)

    # 订阅消息
    client.subscribe(topic)

    client.loop_forever()

def broadcast_thread():
    global Special_Command
    global save_file_path
    global Running_IMU
    global data_buffer
    global Buffer_of_IMU2
    """ Thread function for broadcasting commands at regular intervals. """
    while True:
        # Send a command to all clients
        command_code = input('Input the command for IMU:\n 1 - List All Directory \n 2 - Update Local Time \n 3 - Print Local Time \n 4 - Start Collecting \n 5 - Stop Collecting \n 6 - Connect MQTT Server \n 7 - Export CSV \n 0 - Quit \n')
        
        if command_code == '0':
            message = '{}: {}'.format(nickname, 'stop')
            broadcast(message)
            close_sockets()
        elif command_code == '1':
            message = '{}: {}'.format(nickname, 'listdirectory')
            broadcast(message)
        elif command_code == '2':
            # add the timestamp from the PC and
            message = nickname+":"+"Update_Local_Time_as"+ datetime.datetime.now().strftime("%Y-%m-%d-1-%H-%M-%S-%f")[:-3]
            broadcast(message)
        elif command_code == '3':
            message = '{}: {}'.format(nickname, 'Check_IMU_Time')
            broadcast(message)
        elif command_code == '4':
            Special_Command = "Real_Time_Detection"
            message = '{}: {}'.format(nickname, 'Real_time_Detection_and_Save_as')
            broadcast(message)
        elif command_code == '5':
            if len(data_buffer) > 0 and len(Buffer_of_IMU2) > 0:
                print(len(data_buffer),len(Buffer_of_IMU2))
                Special_Command = ""
                save_file_path = ""
                Running_IMU = None
            message = '{}: {}'.format(nickname, 'stop')# input the command from the console
            broadcast(message)
        elif command_code == '6':
            connect_mqtt_server()
        elif command_code == '7':
            save_recieved_data(data = data_buffer, file_path = save_file_path)
        else:
            print('Invalid Command, please try again.')
            Special_Command = ""
            save_file_path = ""
            Running_IMU = None





# 初始化數據/設定
data_length = 30

data_channels = 12

window_size = 30

x = np.arange(data_length)

y_data = [np.zeros(data_length) for _ in range(data_channels + 1)]

grap_id = 0

record_count = 0

rawData = []

topic = "AD1"

max_data_count = 10000

stop_time_seconds = 100

trained_data = pd.read_csv('../../public/data/csv/a1_1.csv')
trained_data_x = trained_data.iloc[:, 1:-1].values


########### LOAD MODEL ###############
# 初始化一个空列表来保存预测结果
predictions = []
model_path = '../../public/pb/new_test30_1.pb'
x_record = 1
model = load_model(model_path)
########### LOAD MODEL ###############



source1 = ColumnDataSource(data={'x': x, 'y': y_data[1]})
source2 = ColumnDataSource(data={'x': x, 'y': y_data[2]})
source3 = ColumnDataSource(data={'x': x, 'y': y_data[3]})
source4 = ColumnDataSource(data={'x': x, 'y': y_data[4]})
source5 = ColumnDataSource(data={'x': x, 'y': y_data[5]})
source6 = ColumnDataSource(data={'x': x, 'y': y_data[6]})

source7 = ColumnDataSource(data={'x': x, 'y': y_data[7]})
source8 = ColumnDataSource(data={'x': x, 'y': y_data[8]})
source9 = ColumnDataSource(data={'x': x, 'y': y_data[9]})
source10 = ColumnDataSource(data={'x': x, 'y': y_data[10]})
source11 = ColumnDataSource(data={'x': x, 'y': y_data[11]})
source12 = ColumnDataSource(data={'x': x, 'y': y_data[12]})
source13 = ColumnDataSource(data={'x': np.arange(1), 'y': np.zeros(1)})



# 創建 Bokeh 的線型圖
plots = []
plot1 = figure(height=300, title="MPU1", tools="crosshair,pan,reset,save,wheel_zoom")
line_ax1 = plot1.line('x', 'y', source=source1, line_width=2, line_alpha=0.8, line_color='black', legend_label='AX1')
line_ay1 = plot1.line('x', 'y', source=source2, line_width=2, line_alpha=0.8, line_color='blue', legend_label='AY1')
line_az1 = plot1.line('x', 'y', source=source3, line_width=2, line_alpha=0.8, line_color='red', legend_label='AZ1')
line_gx1 = plot1.line('x', 'y', source=source4, line_width=2, line_alpha=0.8, line_color='green', legend_label='GX1')
line_gy1 = plot1.line('x', 'y', source=source5, line_width=2, line_alpha=0.8, line_color='yellow', legend_label='GY1')
line_gz1 = plot1.line('x', 'y', source=source6, line_width=2, line_alpha=0.8, line_color='pink', legend_label='GZ1')



# plot2 = figure(plot_height=300, title="Channel AY1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot2.line('x', 'y', source=source2, line_width=2, line_alpha=0.8)

# plot3 = figure(plot_height=300, title="Channel AZ1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot3.line('x', 'y', source=source3, line_width=2, line_alpha=0.8)

# plot4 = figure(plot_height=300, title="Channel GX1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot4.line('x', 'y', source=source4, line_width=2, line_alpha=0.8)

# plot5 = figure(plot_height=300, title="Channel GY1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot5.line('x', 'y', source=source5, line_width=2, line_alpha=0.8)

# plot6 = figure(plot_height=300, title="Channel GZ1", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot6.line('x', 'y', source=source6, line_width=2, line_alpha=0.8)

plot7 = figure(height=300, title="MPU2", tools="crosshair,pan,reset,save,wheel_zoom")
line_ax2 = plot7.line('x', 'y', source=source7, line_width=2, line_alpha=0.8, line_color='black', legend_label='AX2')
line_ay2 = plot7.line('x', 'y', source=source8, line_width=2, line_alpha=0.8, line_color='blue', legend_label='AY2')
line_az2 = plot7.line('x', 'y', source=source9, line_width=2, line_alpha=0.8, line_color='red', legend_label='AZ2')
line_gx2 = plot7.line('x', 'y', source=source10, line_width=2, line_alpha=0.8, line_color='green', legend_label='GX2')
line_gy2 = plot7.line('x', 'y', source=source11, line_width=2, line_alpha=0.8, line_color='yellow', legend_label='GY2')
line_gz2 = plot7.line('x', 'y', source=source12, line_width=2, line_alpha=0.8, line_color='pink', legend_label='GZ2')


# plot8 = figure(plot_height=300, title="Channel AY2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot8.line('x', 'y', source=source8, line_width=2, line_alpha=0.8)

# plot9 = figure(plot_height=300, title="Channel AZ2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot9.line('x', 'y', source=source9, line_width=2, line_alpha=0.8)

# plot10 = figure(plot_height=300, title="Channel GX2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot10.line('x', 'y', source=source10, line_width=2, line_alpha=0.8)

# plot11 = figure(plot_height=300, title="Channel GY2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot11.line('x', 'y', source=source11, line_width=2, line_alpha=0.8)

# plot12 = figure(plot_height=300, title="Channel GZ2", tools="crosshair,pan,reset,save,wheel_zoom")
# line = plot12.line('x', 'y', source=source12, line_width=2, line_alpha=0.8)

plot13 = figure(height=300, title="Predictions", tools="crosshair,pan,reset,save,wheel_zoom")
line = plot13.line('x', 'y', source=source13, line_width=2, line_alpha=0.8, line_color='black')


plots.append(plot1)
plots.append(plot7)
plots.append(plot13)
# plots.append(plot3)
# plots.append(plot4)
# plots.append(plot5)
# plots.append(plot6)
# plots.append(plot7)
# plots.append(plot8)
# plots.append(plot9)
# plots.append(plot10)
# plots.append(plot11)
# plots.append(plot12)


grid = layout(plots, sizing_mode='scale_width')

output_notebook()


show(grid, notebook_handle=True)


test_mode = True
if not test_mode:
    connect_mqtt_server()

while test_mode:
    test_data = []
    for i in range(30):
        test_data.append([
            '('+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") + ')', 
            random.uniform(1.7, 2),
            random.uniform(6.7, 7),
            random.uniform(-7, -6.6),
            random.uniform(-4, 2),
            random.uniform(-2, 2),
            random.uniform(-8, -3),
            random.uniform(0, 0.3),
            random.uniform(9, 9.3),
            random.uniform(-4.3, -4),
            random.uniform(0, 5),
            random.uniform(-2, 2),
            random.uniform(-6, -4),
        ])
    update(test_data)
    time.sleep(0.3)



Loading BokehJS ...

-----------------
fifo: [[['1.7882007748204651' '6.8678610218129466' '-6.855323237975427'
   '-0.19861305780344685' '1.0219132106742719' '-5.90352968944102'
   '0.2766652814342265' '9.143134021096774' '-4.291626970685319'
   '4.710792725709966' '-0.9794907759748286' '-5.841286376033004']
  ['1.9517328077795215' '6.947296455743066' '-6.845875763823634'
   '-2.749942934850327' '-1.9101987269335639' '-4.318396766055155'
   '0.09561615831829727' '9.067783660258561' '-4.143336902261762'
   '0.9986916930816386' '-1.2812402926161002' '-4.794690461640957']
  ['1.853460675732887' '6.886534958477808' '-6.846825729780319'
   '0.0735302200136676' '-1.1437723631333072' '-5.798912187908604'
   '0.20908266943889622' '9.17484492165888' '-4.238734579573537'
   '3.10912310926748' '0.7453429983104543' '-5.642294440241569']
  ['1.853001117200866' '6.942580764672328' '-6.7253050901869385'
   '1.3233245645673364' '-0.1662100328216254' '-5.003438498458198'
   '0.13770780037231262' '9.136173858361769' '-4.237

2024-01-30 15:00:46.628659: W tensorflow/core/framework/op_kernel.cc:1805] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'sequential/Cast' defined at (most recent call last):
    File "/opt/anaconda3/envs/Python39/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/anaconda3/envs/Python39/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/anaconda3/envs/Python39/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/anaconda3/envs/Python39/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/opt/anaconda3/envs/Python39/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 701, in start
      self.io_loop.start()
    File "/opt/anaconda3/envs/Python39/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/anaconda3/envs/Python39